In [1]:
import cogent3
from cogent3 import get_app
from phylim.apps import phylim
import libs
import paths

In [2]:
folder_in = paths.DATA_APES114 + 'introns_ref/'
in_dstore = cogent3.open_data_store(folder_in, suffix='fa', mode='r')

print(in_dstore.describe)

Directory datastore
record type      number
-----------------------
completed           545
not_completed         0
logs                  1
-----------------------


In [3]:
#perform a codon alignment and remove stop codons
loader = get_app("load_aligned", moltype="dna")
omit_degs = get_app("omit_degenerates", moltype="dna")
rename = libs.renamer_aligned()
concat = get_app("concat", moltype="dna")

app = loader + omit_degs + rename
#By using this renamer_aligned function I throw away sequences with paralogs
results_allpos = [r for r in app.as_completed(in_dstore[:], parallel=True) if r]
alpos_alns = concat(results_allpos)
alpos_alns.source = "introns_alignments"
alpos_alns

   0%|          |00:00<?

,0
Chimpanzee,AGGGATGTGTGTCGGTGTGTGTGGAGAGGGTGCTCCTTCGCTCCTGCTTTCCCGGCTCGG
Gorilla,.........................................................T..
Human,............................................................


In [4]:
#Get distances of concatenated sequences 
#time_het=max sets a 36 params substitution model
sm = get_app("model", "GN", time_het="max", optimise_motif_probs=True, show_progress=True)
result_intergenic = sm(alpos_alns)

checker = get_app("phylim")
checked = checker(result_intergenic)
lengthstree = result_intergenic.lf.get_lengths_as_ens()
print("Is model identifiable? " + str(checked.is_identifiable) + "\n")
print("Model results : \n" + str(result_intergenic.lf) + "\n")
print("Distances: " + str(lengthstree))

   0%|          |00:00<?

   0%|          |00:00<?

Is model identifiable? True

Model results : 
GN
log-likelihood = -11081110.8174
number of free parameters = 39
edge          parent    length     A>C     A>G     A>T     C>A     C>G
----------------------------------------------------------------------
Gorilla       root        0.01    1.01    4.53    0.68    1.18    1.62
Human         root        0.01    1.01    3.85    0.63    1.15    1.50
Chimpanzee    root        0.01    0.99    3.61    0.69    1.05    1.50
----------------------------------------------------------------------

continued: 
 C>T     G>A     G>C     G>T     T>A     T>C
--------------------------------------------
6.30    6.37    1.62    1.17    0.69    4.42
6.32    6.39    1.51    1.15    0.60    3.80
5.96    6.01    1.39    1.04    0.66    3.38
--------------------------------------------

   A       C       G       T
----------------------------
0.25    0.25    0.25    0.25
----------------------------

Distances: {'Gorilla': np.float64(0.010197461794107378), 'Hum